# Advanced Reccomendations

## Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
movie_rating = pd.read_csv('u.data', sep='\t', names=columns)

In [3]:
movie_rating.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [4]:
movies = pd.read_csv('Movie_Id_Titles', sep=',', index_col='item_id')

In [5]:
df = movie_rating.merge(movies, on='item_id')

In [6]:
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [7]:
df['title'].nunique()

1664

In [11]:
df['user_id'].nunique()

944

In [12]:
df.shape

(100003, 5)

# Model Preparation

## Train Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [13]:
train_data, test_data = train_test_split(df, test_size=.25)

In [14]:
train_data.shape, test_data.shape

((75002, 5), (25001, 5))